In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

# ========== 工具函数 ==========
def inputdata(path):
    try:
        return pd.read_csv(path, header=0, sep=",", encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, header=0, sep=",", encoding="gbk")

def outputdata(path, data, is_index=False):
    data.to_csv(path, index=is_index, header=True, sep=",", mode="w", encoding="utf-8")

def transcolname(df, column_mapping):
    return df.rename(columns=column_mapping)

def trans_datetime(df):
    df['Date_str'] = df['Date']
    dt = pd.to_datetime(df['Date_str'], format='%Y-%m-%d')

    df['year'] = dt.dt.year
    df['month'] = dt.dt.month
    df['day'] = dt.dt.day
    df['weekday'] = dt.dt.weekday

    unique_dates = pd.Series(dt.dt.strftime('%Y-%m-%d')).sort_values().unique()
    date_mapping = {date: i + 1 for i, date in enumerate(unique_dates)}
    df['Date'] = dt.dt.strftime('%Y-%m-%d').map(date_mapping)

    df.drop(columns=['Date_str'], inplace=True)
    return df

def augment_features(df):
    df = df.sort_values(['StockCode', 'Date'])

    # 滞后收盘价
    df['PrevClose'] = df.groupby('StockCode')['Close'].shift(1)
    # 日收益率
    df['Return'] = df['Close'] / df['PrevClose'] - 1
    # 高低价差、开收差
    df['HighLowDiff'] = df['High'] - df['Low']
    df['OpenCloseDiff'] = df['Open'] - df['Close']
    # 成交量变化率
    df['VolumePct'] = df.groupby('StockCode')['Volume'].pct_change()

    # 滑动均值与标准差
    for w in (5, 10, 20):
        df[f'MA_{w}'] = df.groupby('StockCode')['Close'].transform(lambda x: x.rolling(w, min_periods=w).mean())
        df[f'STD_{w}'] = df.groupby('StockCode')['Close'].transform(lambda x: x.rolling(w, min_periods=w).std())

    return df.reset_index(drop=True)

def scale_features(df, method='standard', scaler_path=r'C:\Users\27535\Desktop\大数据挑战赛\model\scaler.bin'):
    num_cols = [
        'Open', 'Close', 'High', 'Low',
        'Volume', 'Turnover', 'Amplitude', 'PriceChange', 'TurnoverRate',
        'PrevClose', 'Return', 'HighLowDiff', 'OpenCloseDiff', 'VolumePct'
    ] + [f'MA_{w}' for w in (5, 10, 20)] + [f'STD_{w}' for w in (5, 10, 20)]

    missing_cols = [col for col in num_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"以下列缺失: {missing_cols}")

    if method == 'standard':
        scaler = StandardScaler()
    elif method == 'minmax':
        scaler = MinMaxScaler()
    else:
        raise ValueError("method 必须是 'standard' 或 'minmax'")

    df[num_cols] = scaler.fit_transform(df[num_cols])
    joblib.dump(scaler, scaler_path)
    print(f"Scaler 已保存到 {scaler_path}")
    return df, scaler

def processing_feature():
    # 1. 读入数据并重命名
    data = inputdata(r"C:\Users\27535\Desktop\大数据挑战赛\data\train.csv")
    data = transcolname(data, {
        "股票代码": "StockCode", "日期": "Date", "开盘": "Open", "收盘": "Close",
        "最高": "High", "最低": "Low", "成交量": "Volume", "成交额": "Turnover",
        "振幅": "Amplitude", "涨跌额": "PriceChange", "换手率": "TurnoverRate",
        "涨跌幅": "PriceChangePercentage",
    })
    data.drop(columns=["PriceChangePercentage"], inplace=True)
    # 2. 日期处理
    data = trans_datetime(data)

    # 3. 扩展特征
    data = augment_features(data)

    # 4. 替换 inf/-inf 为 0，再填充 NaN
    data.replace([float('inf'), -float('inf')], 0, inplace=True)
    data.fillna(0, inplace=True)

    return data


feature = processing_feature()
outputdata(r"C:\Users\27535\Desktop\大数据挑战赛\temp\feature.csv", feature, is_index=False)

# # 进行标准缩放
# scaled_feature, scaler = scale_features(
#     feature,
#     method='standard',
#     scaler_path=r"C:\Users\27535\Desktop\大数据挑战赛\model\scaler.bin"
# )

# outputdata(r"C:\Users\27535\Desktop\大数据挑战赛\temp\feature_scaled.csv", scaled_feature, is_index=False)


Scaler 已保存到 C:\Users\27535\Desktop\大数据挑战赛\model\scaler.bin


In [19]:


import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import spearmanr
import numpy as np

def inputdata(path):
    data = pd.read_csv(path, header=0, sep=",", encoding="utf-8")
    return data

# 加载特征数据
feature = inputdata(r"C:\Users\27535\Desktop\大数据挑战赛\temp\feature.csv")

# 数据处理函数
def process_data(npdf, stp=32):
    ret = []
    for i in range(npdf.shape[0] - stp):
        train_seq = npdf[i : i + stp]
        train_label = npdf[i + stp]
        train_seq = torch.FloatTensor(train_seq)
        train_label = torch.FloatTensor(train_label).view(-1)
        ret.append((train_seq, train_label))
    return ret

# 准备数据
column_names = feature.columns.tolist()
stockcodes = feature["StockCode"].drop_duplicates().tolist()

train_data = []
for stockcode in stockcodes:
    stock_data = feature[feature["StockCode"] == stockcode]
    max_date = stock_data["Date"].max()
    min_date = stock_data["Date"].min()
    stock_data = stock_data.values
    if len(stock_data) < 32:
        continue
    train_data += process_data(stock_data, stp=32)

input_size = len(column_names)
d_model = 512
n_heads = 8
e_layers = 2
dropout = 0.2
output_size = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Transformer(nn.Module):
    def __init__(self, input_size, d_model, n_heads, e_layers, output_size, dropout):
        super(Transformer, self).__init__()
        self.input_embedding = nn.Linear(input_size, d_model)
        self.embedding_ln    = nn.LayerNorm(d_model)
        self.positional_encoding = self.create_positional_encoding(seq_len=32, d_model=d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=e_layers) 
        self.fc = nn.Linear(d_model, output_size)

    def create_positional_encoding(self, seq_len, d_model):
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.to(device)

    def forward(self, x):
        # x 形状：(batch_size, seq_len, input_size)
        batch_size, seq_len, _ = x.size()
        x = self.input_embedding(x)      # (B, L, d_model)
        x = self.embedding_ln(x)
        pe = self.positional_encoding[:seq_len, :].unsqueeze(0).expand(batch_size, -1, -1)
        x = x + pe  # 添加位置编码
        x = self.transformer_encoder(x)  # 使用 transformer_encoder
        out = self.fc(x[:, -1, :])  # 取最后一个时间步：(batch_size, output_size)
        return out

def custom_loss(outputs, targets, f1_max_weight=0.2, spearman_max_weight=0.3, f1_min_weight=0.2, spearman_min_weight=0.3, k_ratio=0.1):
    batch_size = targets.size(0)
    k = max(1, int(batch_size * k_ratio))  # 最大/最小回报的样本数（例如批次的 10%）

    # 重塑 outputs 和 targets
    outputs = outputs.view(-1)  # (batch_size,)
    targets = targets.view(-1)  # (batch_size,)

    # 按回报值对 targets 排序，识别最大和最小回报
    _, indices = torch.sort(targets, descending=True)
    max_indices = indices[:k]  # 前 k 个样本（最大回报）
    min_indices = indices[-k:]  # 后 k 个样本（最小回报）

    # 辅助函数：计算平滑的 F1 分数
    def compute_f1(pred, true):
        # 使用 sigmoid 平滑二值化
        pred_prob = torch.sigmoid(pred)  # 将预测值映射到 (0,1)
        true_binary = (true > 0).float()  # 真实值仍使用硬二值化
        true_positives = (pred_prob * true_binary).sum()
        predicted_positives = pred_prob.sum()
        actual_positives = true_binary.sum()
        precision = true_positives / (predicted_positives + 1e-8)
        recall = true_positives / (actual_positives + 1e-8)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        return f1

    # 辅助函数：计算斯皮尔曼相关系数的可微近似
    def compute_spearman(pred, true):
        if len(pred) <= 1:
            return torch.tensor(0.0, device=pred.device, requires_grad=True)

        # 使用简单的相关系数近似，避免复杂的排名操作
        pred_norm = pred - pred.mean()
        true_norm = true - true.mean()
        cov = (pred_norm * true_norm).sum()
        pred_std = torch.sqrt((pred_norm ** 2).sum() + 1e-8)
        true_std = torch.sqrt((true_norm ** 2).sum() + 1e-8)
        pearson_corr = cov / (pred_std * true_std + 1e-8)
        return pearson_corr  # 使用皮尔逊相关系数作为斯皮尔曼的近似

    # 计算最大回报的 F1 和相关系数
    f1_max = compute_f1(outputs[max_indices], targets[max_indices])
    spearman_max = compute_spearman(outputs[max_indices], targets[max_indices])

    # 计算最小回报的 F1 和相关系数
    f1_min = compute_f1(outputs[min_indices], targets[min_indices])
    spearman_min = compute_spearman(outputs[min_indices], targets[min_indices])

    # 组合损失
    loss = (
        f1_max_weight * (1 - f1_max) +
        spearman_max_weight * (1 - spearman_max) +
        f1_min_weight * (1 - f1_min) +
        spearman_min_weight * (1 - spearman_min)
    )
    return 1/loss
from tqdm import tqdm
import time
def train_model(train_data, i, num_epochs=10 ):
    if len(train_data) == 0:
        return Transformer(input_size, d_model, n_heads, e_layers, output_size, dropout).to(device)

    train_data = [(x.to(device), y.to(device)) for x, y in train_data]

    X_train_tensor = torch.stack([x for x, _ in train_data])
    y_train_tensor = torch.stack([y[i] for _, y in train_data])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    model = Transformer(
        input_size=input_size,
        d_model=d_model,
        n_heads=n_heads,
        e_layers=e_layers,
        output_size=output_size,
        dropout=dropout
    ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

    print(f"开始训练，样本数：{len(train_dataset)}, 批次数：{len(train_loader)}")
    total_start = time.time()
    criterion = nn.MSELoss()
    for epoch in tqdm(range(num_epochs), desc="训练进度"):
        epoch_start = time.time()
        tot_loss = 0.0
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            tot_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
        scheduler.step()
        epoch_time = time.time() - epoch_start
        print(f"轮次 [{epoch + 1}/{num_epochs}] loss: {tot_loss / len(train_loader):.4f} - 耗时: {epoch_time:.2f}s")

    total_time = time.time() - total_start
    print(f"训练完成，总耗时：{total_time:.2f} 秒")
    return model

# 训练模型以预测
colname2index = {x: i for i, x in enumerate(column_names)}
model_i = train_model(train_data, colname2index["Close"] + 2, num_epochs=10 )

# 保存模型
model_name = r"C:\Users\27535\Desktop\大数据挑战赛\model\model_Close.bin"
pickle.dump(model_i, open(model_name, "wb"))

开始训练，样本数：616529, 批次数：9634


训练进度:  10%|█         | 1/10 [04:52<43:54, 292.69s/it]

轮次 [1/10] loss: 7626.1672 - 耗时: 292.69s


训练进度:  20%|██        | 2/10 [09:49<39:23, 295.38s/it]

轮次 [2/10] loss: 7624.2605 - 耗时: 297.26s


...

轮次 [8/10] loss: 7623.5405 - 耗时: 296.02s


训练进度:  90%|█████████ | 9/10 [44:22<04:55, 295.27s/it]

轮次 [9/10] loss: 7624.2012 - 耗时: 293.19s


训练进度: 100%|██████████| 10/10 [49:14<00:00, 295.47s/it]

轮次 [10/10] loss: 7624.4944 - 耗时: 292.64s
训练完成，总耗时：2954.67 秒


In [24]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import joblib

pred_len = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. 与训练时完全一致的：重命名、时间拆分、特征扩增、缩放
def inputdata(path):
    return pd.read_csv(path, header=0, sep=",", encoding="utf-8")

def transcolname(df, mapping):
    return df.rename(columns=mapping)

def trans_datetime(df):
    df = df.copy()
    df["Date_str"] = df["Date"]
    dt = pd.to_datetime(df["Date_str"], format="%Y-%m-%d")
    df["year"]    = dt.dt.year
    df["month"]   = dt.dt.month
    df["day"]     = dt.dt.day
    df["weekday"] = dt.dt.weekday
    unique_dates = dt.dt.strftime("%Y-%m-%d").sort_values().unique()
    date_map = {d: i+1 for i, d in enumerate(unique_dates)}
    df["Date"] = dt.dt.strftime("%Y-%m-%d").map(date_map)
    df.drop(columns=["Date_str"], inplace=True)
    return df

def augment_features(df):
    df = df.sort_values(["StockCode","Date"]).reset_index(drop=True)
    df["PrevClose"]      = df.groupby("StockCode")["Close"].shift(1)
    df["Return"]         = df["Close"] / df["PrevClose"] - 1
    df["HighLowDiff"]    = df["High"] - df["Low"]
    df["OpenCloseDiff"]  = df["Open"] - df["Close"]
    df["VolumePct"]      = df.groupby("StockCode")["Volume"].pct_change()
    for w in (5,10,20):
        df[f"MA_{w}"]  = df.groupby("StockCode")["Close"] \
                            .transform(lambda x: x.rolling(w,min_periods=1).mean())
        df[f"STD_{w}"] = df.groupby("StockCode")["Close"] \
                            .transform(lambda x: x.rolling(w,min_periods=1).std())
    return df

def scale_features(df):
    # 与训练时完全相同的数值列顺序
    num_cols = [
        "Open","Close","High","Low",
        "Volume","Turnover","Amplitude","PriceChange","TurnoverRate",
        "PrevClose","Return","HighLowDiff","OpenCloseDiff","VolumePct"
    ] + [f"MA_{w}" for w in (5,10,20)] + [f"STD_{w}" for w in (5,10,20)]

    return df

# 2. 载入训练时保存的 scaler 与模型
# scaler = joblib.load(open(r"C:\Users\27535\Desktop\大数据挑战赛\model\scaler.bin","rb"))
model = pickle.load(open(r"C:\Users\27535\Desktop\大数据挑战赛\model\model_Close.bin","rb"))
model.to(device).eval()

# 3. 测试数据处理函数
def processing_feature_test(test_csv):
    mapping = {
        "股票代码":"StockCode","日期":"Date","开盘":"Open","收盘":"Close",
        "最高":"High","最低":"Low","成交量":"Volume","成交额":"Turnover",
        "振幅":"Amplitude","涨跌额":"PriceChange","换手率":"TurnoverRate",
        "涨跌幅":"PriceChangePercentage"
    }
    df = inputdata(test_csv)
    df = transcolname(df, mapping)
    df = trans_datetime(df)
    df = augment_features(df)

    # ==== 清除 inf 和 NaN，必须在缩放前 ====
    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.fillna(0, inplace=True)

    df = scale_features(df)

    # 只保留最后 pred_len 天
    max_date = df["Date"].max()
    df = df[df["Date"] > max_date - pred_len]
    return df


# 4. 加载并处理 test.csv
test_df = processing_feature_test(r"C:\Users\27535\Desktop\大数据挑战赛\data\test.csv")
feature_cols = [c for c in test_df.columns if c != "Date"]

# 5. 按股票逐个预测
results = []
for sc in test_df["StockCode"].unique():
    sub = test_df[test_df["StockCode"]==sc].copy()
    if len(sub) < pred_len:
        continue  # 数据不够
    seq = sub[feature_cols].values[-pred_len:]          # (32, feat_dim)
    inp = torch.tensor(seq, dtype=torch.float32)       \
               .unsqueeze(0).to(device)                # (1,32,feat_dim)
    with torch.no_grad():
        pred = model(inp)                              # (1,1)
    pred = pred.cpu().item()
    # 找到最后一天的真实收盘价
    last_close = sub[sub["Date"]==sub["Date"].max()]["Close"].values[0]
    pct_change = (pred - last_close) / last_close * 100
    results.append((sc, pct_change))

# 6. 取涨幅最大/最小10只
results = sorted(results, key=lambda x: x[1], reverse=True)
top10_up   = [r[0] for r in results[:10]]
top10_down = [r[0] for r in results[-10:]]

# 7. 保存
out = pd.DataFrame({
    "涨幅最大股票代码": top10_up,
    "涨幅最小股票代码": top10_down
})
out.to_csv(r"C:\Users\27535\Desktop\大数据挑战赛\output\result_transformer.csv",
           index=False, encoding="utf-8")
print("预测完毕，结果保存在 output/result_transformer.csv")


预测完毕，结果保存在 output/result_transformer.csv
